In [ ]:
# 🚀 앙상블 기법을 활용한 한국어 텍스트 분류 최적화

이 노트북에서는 다음을 수행합니다:
1. **앙상블 하이퍼파라미터 최적화**: 다양한 조합을 테스트하여 최적값 탐색
2. **V3 앙상블 적용**: NER + 키워드 평균 임베딩에 앙상블 기법 적용
3. **V4 파인튜닝 + 앙상블**: 파인튜닝된 모델에 앙상블 기법 적용

## 📊 데이터 사용량
- **전체 데이터 활용**: 1,447개의 실제 한국어 텍스트 데이터 사용
- **앙상블 평가**: 전체 데이터로 정확한 성능 측정
- **파인튜닝**: 800개 데이터로 3 에포크 학습
- **하이퍼파라미터 최적화**: 10가지 가중치 조합 테스트

## 📋 실행 전 준비사항
1. 데이터셋 업로드 (아래 셀에서 안내)
2. 패키지 설치 및 환경 설정
3. 하드웨어 가속 설정 (GPU 권장)


In [ ]:
# 🔧 환경 설정 및 패키지 설치
!pip install sentence-transformers transformers torch scikit-learn pandas numpy tqdm matplotlib seaborn

# 하드웨어 확인
import torch
print(f"🔍 하드웨어 정보:")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 장치: {torch.cuda.get_device_name()}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
print(f"CPU 코어 수: {torch.get_num_threads()}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎯 사용할 디바이스: {device}")

# 기본 설정
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# wandb 비활성화 (로그인 없이 실행하기 위함)
os.environ['WANDB_DISABLED'] = 'true'
print("🔕 wandb 추적 비활성화됨 (로그인 불필요)")


In [ ]:
## 📁 데이터셋 준비 방법

### 방법 1: 직접 업로드
1. 왼쪽 패널의 파일 아이콘 클릭
2. `data.csv` 파일을 드래그 앤 드롭으로 업로드

### 방법 2: Google Drive 연동
```python
from google.colab import drive
drive.mount('/content/drive')
# 파일 경로: /content/drive/MyDrive/your_file.csv
```

### 방법 3: 샘플 데이터 생성 (테스트용)
아래 셀에서 샘플 데이터를 생성할 수 있습니다.


In [ ]:
# 📊 샘플 데이터 생성 (실제 데이터가 없는 경우)
import pandas as pd
import numpy as np

def create_sample_data():
    """테스트용 샘플 데이터 생성"""
    np.random.seed(42)
    
    # 카테고리별 샘플 텍스트
    sample_data = {
        '학업': [
            '수학 과제 제출하기', '영어 시험 공부하기', '논문 작성하고 발표하기', 
            '프로젝트 팀 회의 참석하기', '실험실에서 연구하기', '도서관에서 자료 조사하기'
        ],
        '업무': [
            '회의 참석하고 보고서 작성하기', '클라이언트와 미팅하기', '프레젠테이션 준비하기',
            '메일 확인하고 답변하기', '새로운 프로젝트 기획하기', '팀원들과 협업하기'
        ],
        '건강': [
            '헬스장에서 운동하기', '병원에서 건강 검진 받기', '요가 클래스 참여하기',
            '산책하며 스트레스 해소하기', '충분한 수면 취하기', '건강한 식단 관리하기'
        ],
        '경제': [
            '가계부 정리하고 예산 관리하기', '투자 포트폴리오 점검하기', '적금 넣기',
            '부동산 시장 조사하기', '재테크 강의 듣기', '용돈 기입장 작성하기'
        ],
        '친목': [
            '친구들과 카페에서 만나기', '동창회 참석하기', '가족과 저녁 식사하기',
            '동료들과 회식하기', '커뮤니티 모임 참여하기', '새로운 사람들과 네트워킹하기'
        ],
        '취미': [
            '독서하며 여가 시간 보내기', '영화 관람하기', '요리 레시피 도전하기',
            '여행 계획 세우기', '취미 클래스 수강하기', '온라인 게임 즐기기'
        ]
    }
    
    # 데이터프레임 생성
    rows = []
    for category, titles in sample_data.items():
        for title in titles:
            rows.append({'title': title, 'categories': category})
    
    # 추가 랜덤 데이터 생성
    categories = list(sample_data.keys())
    for i in range(100):  # 100개 추가 샘플
        category = np.random.choice(categories)
        title = f"{category} 관련 활동 {i+1}"
        rows.append({'title': title, 'categories': category})
    
    df = pd.DataFrame(rows)
    df.to_csv('sample_data.csv', index=False)
    print(f"✅ 샘플 데이터 생성 완료: {len(df)}개 항목")
    return df

# 실제 데이터 파일이 있는지 확인
DATA_PATH = 'data/data.csv'  # 실제 데이터 경로
if not os.path.exists(DATA_PATH):
    # 대체 경로 확인
    if os.path.exists('data.csv'):
        DATA_PATH = 'data.csv'
    else:
        print("❌ data.csv 파일을 찾을 수 없습니다.")
        print("🔄 샘플 데이터를 생성합니다...")
        df = create_sample_data()
        DATA_PATH = 'sample_data.csv'

if 'df' not in locals():
    print(f"✅ {DATA_PATH} 파일을 찾았습니다.")
    df = pd.read_csv(DATA_PATH)
    print(f"📊 전체 데이터: {len(df)}개 항목 로드")

print(f"📊 데이터 정보: {len(df)}개 행, {len(df.columns)}개 열")
print(df.head())


In [ ]:
# 🔧 설정 및 유틸리티 함수
from sentence_transformers import SentenceTransformer, util, InputExample, losses, evaluation
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm
import random
import time
import json
from collections import defaultdict
from itertools import product

# 모델 및 설정
BASE_MODEL_NAME = 'jhgan/ko-sroberta-multitask'
NER_MODEL_NAME = 'Leo97/KoELECTRA-small-v3-modu-ner'

# 카테고리 정의 (키워드 기반)
CATEGORIES_DEFINITIONS = {
    '학업': [
        '과제 작성하고 제출하기', '시험 준비하고 공부하기', '연구 활동하고 논문 쓰기',
        '실험실에서 연구하기', '세미나 참석하고 발표하기', '도서관에서 자료 찾기',
        '프로젝트 팀원과 협업하기', '온라인 강의 수강하기'
    ],
    '업무': [
        '회의 참석하고 의견 제시하기', '보고서 작성하고 검토하기', '클라이언트와 미팅하기',
        '프레젠테이션 준비하고 발표하기', '업무 메일 확인하고 답변하기', '새 프로젝트 기획하기',
        '동료들과 협업하고 소통하기', '실적 분석하고 개선하기'
    ],
    '건강': [
        '헬스장에서 운동하고 근력 기르기', '병원에서 건강 검진 받기', '요가나 필라테스 하기',
        '산책하며 스트레스 해소하기', '충분한 수면 취하고 컨디션 관리하기', '건강한 식단 계획하고 실천하기',
        '금연과 금주 실천하기', '정기적인 건강 관리하기'
    ],
    '경제': [
        '가계부 정리하고 지출 관리하기', '투자 포트폴리오 구성하고 관리하기', '적금과 예금 넣기',
        '부동산 시장 조사하고 분석하기', '재테크 공부하고 실천하기', '용돈 기입장 작성하기',
        '보험 상품 비교하고 가입하기', '세금 신고하고 환급받기'
    ],
    '친목': [
        '친구들과 만나서 대화하기', '동창회나 모임 참석하기', '가족과 시간 보내고 소통하기',
        '동료들과 회식하고 친목 도모하기', '새로운 사람들과 네트워킹하기', '커뮤니티 활동 참여하기',
        '사회적 관계 유지하고 발전시키기', '지인들과 연락하고 안부 묻기'
    ],
    '취미': [
        '독서하며 지식 쌓고 여가 즐기기', '영화나 드라마 시청하기', '요리 레시피 도전하고 음식 만들기',
        '여행 계획 세우고 떠나기', '취미 클래스 수강하고 새 기술 배우기', '게임하며 스트레스 풀기',
        '음악 듣고 악기 연주하기', '그림 그리고 창작 활동하기'
    ]
}

# NER 관련 설정
NER_SPECIAL_TOKENS = ["<PERSON>", "<LOCATION>", "<ORG>"]
V2_IMPROVED_ENTITIES = ["PS", "LC", "OG"]  # 인물, 장소, 조직
NER_CONFIDENCE_THRESHOLD = 0.8

print("✅ 설정 및 카테고리 정의 완료")
print(f"📝 총 {len(CATEGORIES_DEFINITIONS)}개 카테고리:")
for cat, keywords in CATEGORIES_DEFINITIONS.items():
    print(f"  - {cat}: {len(keywords)}개 키워드")


In [ ]:
# 🧠 NER 모델 로드 및 텍스트 전처리 함수
def setup_ner_model():
    """NER 모델을 설정하고 반환"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(NER_MODEL_NAME)
        model = AutoModelForTokenClassification.from_pretrained(NER_MODEL_NAME)
        ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)
        print("✅ NER 모델 로드 완료")
        return ner_pipeline
    except Exception as e:
        print(f"❌ NER 모델 로드 실패: {e}")
        return None

def ner_generalize_texts(texts, entities_to_generalize=V2_IMPROVED_ENTITIES, confidence_threshold=NER_CONFIDENCE_THRESHOLD):
    """NER을 사용하여 텍스트 일반화"""
    ner_pipeline = setup_ner_model()
    if not ner_pipeline:
        return texts  # NER 실패시 원본 반환
    
    generalized_texts = []
    
    for text in tqdm(texts, desc="NER 일반화 진행"):
        try:
            # NER 수행
            entities = ner_pipeline(text)
            
            # 신뢰도 기준으로 필터링
            filtered_entities = [
                entity for entity in entities
                if entity['score'] >= confidence_threshold and 
                entity['entity'].replace('B-', '').replace('I-', '') in entities_to_generalize
            ]
            
            # 엔티티를 위치 순으로 정렬 (뒤에서부터 치환하기 위해 역순)
            filtered_entities.sort(key=lambda x: x['start'], reverse=True)
            
            # 텍스트 치환
            generalized_text = text
            for entity in filtered_entities:
                entity_type = entity['entity'].replace('B-', '').replace('I-', '')
                if entity_type == 'PS':
                    replacement = '<PERSON>'
                elif entity_type == 'LC':
                    replacement = '<LOCATION>'
                elif entity_type == 'OG':
                    replacement = '<ORG>'
                else:
                    continue
                
                generalized_text = (
                    generalized_text[:entity['start']] + 
                    replacement + 
                    generalized_text[entity['end']:]
                )
            
            generalized_texts.append(generalized_text)
            
        except Exception as e:
            # 오류 발생시 원본 텍스트 사용
            generalized_texts.append(text)
    
    return generalized_texts

print("✅ NER 전처리 함수 정의 완료")


In [ ]:
# 🎯 앙상블 최적화 클래스
class EnsembleOptimizer:
    """앙상블 기법의 하이퍼파라미터를 최적화하는 클래스"""
    
    def __init__(self, data_path, sample_size=None):
        self.data_path = data_path
        self.sample_size = sample_size  # None이면 전체 데이터 사용
        
        # 카테고리 정의 (가장 먼저 설정)
        self.categories = list(CATEGORIES_DEFINITIONS.keys())
        
        # 모델 로드
        print("🔄 모델 로딩 중...")
        self.base_model = SentenceTransformer(BASE_MODEL_NAME, device=device)
        
        # 데이터 준비
        self.test_df = self._prepare_data()
        
        # 기본 임베딩들 미리 계산
        self._precompute_embeddings()
        
    def _prepare_data(self):
        """데이터 로드 및 전처리"""
        df = pd.read_csv(self.data_path)
        
        # 카테고리 컬럼 정규화
        if 'categories' not in df.columns and 'category' in df.columns:
            df = df.rename(columns={'category': 'categories'})
        
        df.dropna(subset=['title', 'categories'], inplace=True)
        df['category'] = df['categories'].apply(lambda x: x.split(';')[0].strip() if isinstance(x, str) else x)
        df = df[df['category'].isin(self.categories)]
        
        # 샘플링 (sample_size가 None이면 전체 데이터 사용)
        if self.sample_size is not None and len(df) > self.sample_size:
            df = df.sample(n=self.sample_size, random_state=42)
            print(f"⚡ 빠른 실행을 위해 {self.sample_size}개 샘플로 제한")
        else:
            print(f"📊 전체 데이터 사용: {len(df)}개")
        
        # NER 전처리
        print("🔄 NER 전처리 중...")
        df = df.copy()
        df['generalized_title'] = ner_generalize_texts(df['title'].tolist())
        
        print(f"✅ 테스트 데이터 준비 완료: {len(df)}개")
        return df
    
    def _precompute_embeddings(self):
        """기본 임베딩들을 미리 계산"""
        print("🔄 기본 임베딩 계산 중...")
        
        # 단순 카테고리명 임베딩
        self.simple_cat_embs = self.base_model.encode(
            self.categories, 
            convert_to_tensor=True, 
            normalize_embeddings=True
        )
        
        # 키워드 평균 임베딩
        self.keyword_avg_embs = self._get_keyword_avg_embs()
        
        print("✅ 기본 임베딩 계산 완료")
    
    def _get_keyword_avg_embs(self):
        """카테고리별 키워드 평균 임베딩 계산"""
        category_embs = {}
        for category, keywords in CATEGORIES_DEFINITIONS.items():
            keyword_embs = self.base_model.encode(keywords, convert_to_numpy=True, normalize_embeddings=True)
            avg_emb = np.mean(keyword_embs, axis=0)
            if np.linalg.norm(avg_emb) > 0:
                avg_emb = avg_emb / np.linalg.norm(avg_emb)
            category_embs[category] = avg_emb
        return torch.tensor(np.array(list(category_embs.values()))).to(device)

print("✅ 앙상블 최적화 클래스 정의 완료")


In [ ]:
# 🔍 앙상블 평가 메소드들 추가 (기존 클래스에 메소드 추가)
def _ensemble_predict(self, ensemble_configs, text_column='generalized_title'):
    """앙상블 예측 수행"""
    all_similarities = []
    weights = []
    
    for config in ensemble_configs:
        model_type = config['model']
        embedding_type = config['embedding']
        weight = config['weight']
        
        # 모델 선택 (여기서는 base 모델만 사용)
        model = self.base_model
        
        # 임베딩 타입에 따른 카테고리 임베딩 선택
        if embedding_type == 'simple':
            category_embs = self.simple_cat_embs
        elif embedding_type == 'keyword_avg':
            category_embs = self.keyword_avg_embs
        else:
            continue
            
        # 텍스트 임베딩 계산
        text_embs = model.encode(
            self.test_df[text_column].tolist(), 
            convert_to_tensor=True, 
            normalize_embeddings=True
        )
        
        # 유사도 계산
        similarities = util.cos_sim(text_embs, category_embs).cpu().numpy()
        all_similarities.append(similarities)
        weights.append(weight)
    
    if not all_similarities:
        return None
    
    # 가중 평균으로 앙상블
    weights = np.array(weights)
    weights = weights / weights.sum()  # 정규화
    
    ensemble_similarities = np.zeros_like(all_similarities[0])
    for sim, w in zip(all_similarities, weights):
        ensemble_similarities += sim * w
        
    return ensemble_similarities

def _evaluate_ensemble(self, ensemble_configs, text_column='generalized_title'):
    """앙상블 성능 평가"""
    similarities = self._ensemble_predict(ensemble_configs, text_column)
    if similarities is None:
        return {}
    
    # 예측 결과 계산
    pred_indices = np.argsort(similarities, axis=1)[:, ::-1]  # 내림차순 정렬
    
    true_categories = self.test_df['category'].tolist()
    true_indices = [self.categories.index(cat) for cat in true_categories]
    
    # Hit Rate 계산
    correct_at_1 = sum(1 for i, true_idx in enumerate(true_indices) if true_idx == pred_indices[i, 0])
    correct_at_3 = sum(1 for i, true_idx in enumerate(true_indices) if true_idx in pred_indices[i, :3])
    
    total_count = len(self.test_df)
    hit_rate_1 = correct_at_1 / total_count
    hit_rate_3 = correct_at_3 / total_count
    
    # F1-score 계산
    top_1_predictions = [self.categories[idx] for idx in pred_indices[:, 0]]
    _, _, f1, _ = precision_recall_fscore_support(
        true_categories, top_1_predictions, average='macro', zero_division=0
    )
    
    return {
        'hit_rate_1': hit_rate_1,
        'hit_rate_3': hit_rate_3,
        'f1_score': f1
    }

# 클래스에 메소드 추가
EnsembleOptimizer._ensemble_predict = _ensemble_predict
EnsembleOptimizer._evaluate_ensemble = _evaluate_ensemble

print("✅ 앙상블 평가 메소드 추가 완료")


In [ ]:
# 🔧 하이퍼파라미터 최적화 메소드 (기존 클래스에 메소드 추가)
def optimize_hyperparameters(self):
    """앙상블 하이퍼파라미터 최적화"""
    print("🚀 앙상블 하이퍼파라미터 최적화 시작")
    
    # 최적화할 하이퍼파라미터 정의 (전체 데이터 활용으로 확장)
    ensemble_sizes = [2, 3]  # 앙상블 구성요소 개수
    weight_combinations = [
        # 2-way 앙상블
        [0.5, 0.5],
        [0.6, 0.4],
        [0.7, 0.3],
        [0.8, 0.2],
        [0.4, 0.6],
        [0.3, 0.7],
        # 3-way 앙상블 (향후 확장용)
        [0.4, 0.3, 0.3],
        [0.5, 0.3, 0.2],
        [0.6, 0.3, 0.1],
        [0.33, 0.33, 0.34]
    ]
    
    # 기본 구성요소들
    base_components = [
        {'model': 'base', 'embedding': 'simple'},
        {'model': 'base', 'embedding': 'keyword_avg'}
    ]
    
    best_results = {}
    best_configs = {}
    
    print(f"📊 총 {len(weight_combinations)}개 조합 테스트")
    
    for i, weights in enumerate(weight_combinations):
        ensemble_size = len(weights)
        if ensemble_size > len(base_components):
            continue
            
        # 앙상블 구성 생성
        ensemble_config = []
        for j in range(ensemble_size):
            config = base_components[j].copy()
            config['weight'] = weights[j]
            ensemble_config.append(config)
        
        # 평가 수행
        print(f"\\n🔄 조합 {i+1}/{len(weight_combinations)} 테스트 중...")
        print(f"   구성: {ensemble_config}")
        
        results = self._evaluate_ensemble(ensemble_config)
        if not results:
            continue
        
        # 최고 성능 업데이트
        for metric in ['hit_rate_1', 'hit_rate_3', 'f1_score']:
            if metric not in best_results or results[metric] > best_results[metric]:
                best_results[metric] = results[metric]
                best_configs[metric] = {
                    'config': ensemble_config,
                    'results': results
                }
        
        print(f"   결과: Hit@1={results['hit_rate_1']:.3f}, Hit@3={results['hit_rate_3']:.3f}, F1={results['f1_score']:.3f}")
    
    return best_configs, best_results

def run_baseline_comparison(self):
    """기본 버전들과 성능 비교"""
    print("\\n📊 기본 버전들 성능 측정")
    
    baselines = {}
    
    # V1: 단순 카테고리명 매칭
    print("🔄 V1 (단순 카테고리명) 평가 중...")
    baselines['V1'] = self._evaluate_single_method('simple', 'title')
    
    # V2: NER + 단순 카테고리명
    print("🔄 V2 (NER + 단순 카테고리명) 평가 중...")
    baselines['V2'] = self._evaluate_single_method('simple', 'generalized_title')
    
    # V3: NER + 키워드 평균
    print("🔄 V3 (NER + 키워드 평균) 평가 중...")
    baselines['V3'] = self._evaluate_single_method('keyword_avg', 'generalized_title')
    
    return baselines

def _evaluate_single_method(self, embedding_type, text_column):
    """단일 방법 평가"""
    config = [{'model': 'base', 'embedding': embedding_type, 'weight': 1.0}]
    return self._evaluate_ensemble(config, text_column)

# 클래스에 메소드 추가
EnsembleOptimizer.optimize_hyperparameters = optimize_hyperparameters
EnsembleOptimizer.run_baseline_comparison = run_baseline_comparison
EnsembleOptimizer._evaluate_single_method = _evaluate_single_method

print("✅ 하이퍼파라미터 최적화 메소드 추가 완료")


In [ ]:
# 📊 결과 시각화 및 출력 함수
import matplotlib.pyplot as plt
import seaborn as sns

def plot_results(baselines, best_ensemble_results, title="성능 비교"):
    """결과를 시각화"""
    plt.figure(figsize=(12, 8))
    
    # 데이터 준비
    methods = list(baselines.keys()) + ['Ensemble (Best)']
    hit_rate_1_scores = [baselines[method]['hit_rate_1'] for method in baselines.keys()]
    hit_rate_3_scores = [baselines[method]['hit_rate_3'] for method in baselines.keys()]
    f1_scores = [baselines[method]['f1_score'] for method in baselines.keys()]
    
    # 최고 앙상블 성능 추가
    if 'hit_rate_1' in best_ensemble_results:
        hit_rate_1_scores.append(best_ensemble_results['hit_rate_1'])
        hit_rate_3_scores.append(best_ensemble_results['hit_rate_3'])
        f1_scores.append(best_ensemble_results['f1_score'])
    
    # 서브플롯 생성
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Hit Rate @1
    axes[0].bar(methods, hit_rate_1_scores, color='skyblue', alpha=0.8)
    axes[0].set_title('Hit Rate @1')
    axes[0].set_ylabel('Accuracy')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Hit Rate @3
    axes[1].bar(methods, hit_rate_3_scores, color='lightcoral', alpha=0.8)
    axes[1].set_title('Hit Rate @3')
    axes[1].set_ylabel('Accuracy')
    axes[1].tick_params(axis='x', rotation=45)
    
    # F1 Score
    axes[2].bar(methods, f1_scores, color='lightgreen', alpha=0.8)
    axes[2].set_title('F1 Score (Macro)')
    axes[2].set_ylabel('F1 Score')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.suptitle(title, y=1.02)
    plt.show()

def print_detailed_results(baselines, best_configs, best_results):
    """상세 결과 출력"""
    print("\\n" + "="*80)
    print(" " * 25 + "🏆 최종 성능 비교 결과")
    print("="*80)
    
    # 기본 버전들 결과
    print("\\n📈 기본 버전들 성능:")
    for method, results in baselines.items():
        print(f"  {method:15} | Hit@1: {results['hit_rate_1']:.3f} | Hit@3: {results['hit_rate_3']:.3f} | F1: {results['f1_score']:.3f}")
    
    # 최고 앙상블 결과
    print("\\n🔥 최고 앙상블 성능:")
    for metric, value in best_results.items():
        print(f"  {metric:15} | {value:.3f}")
    
    # 최적 설정 출력
    print("\\n⚙️ 최적 앙상블 설정:")
    for metric, config_info in best_configs.items():
        print(f"\\n📊 {metric} 최적 설정:")
        print(f"   구성: {config_info['config']}")
        print(f"   성능: {config_info['results']}")

print("✅ 시각화 및 출력 함수 정의 완료")


In [ ]:
# 🚀 V3 앙상블 최적화 실행
print("="*80)
print(" " * 20 + "🔬 V3 앙상블 기법 최적화 실험")
print("="*80)

# 앙상블 최적화 인스턴스 생성 (전체 데이터 사용)
optimizer = EnsembleOptimizer(DATA_PATH, sample_size=None)  # 전체 데이터 사용

# 기본 버전들 성능 측정
print("\\n1️⃣ 기본 버전들(V1, V2, V3) 성능 측정")
baselines = optimizer.run_baseline_comparison()

# 앙상블 하이퍼파라미터 최적화
print("\\n\\n2️⃣ 앙상블 하이퍼파라미터 최적화")
best_configs, best_results = optimizer.optimize_hyperparameters()

# 결과 시각화
print("\\n\\n3️⃣ 결과 시각화")
plot_results(baselines, best_results, "V3 vs 앙상블 성능 비교")

# 상세 결과 출력
print_detailed_results(baselines, best_configs, best_results)


In [ ]:
## 🔧 V4: 파인튜닝 + 앙상블 기법

V3의 앙상블 결과가 좋다면, 이제 파인튜닝을 추가로 적용해보겠습니다.


In [ ]:
# 🎓 파인튜닝 클래스 정의
from torch.utils.data import DataLoader

class QuickFinetuner:
    """한국어 텍스트 분류를 위한 파인튜닝 클래스"""
    
    def __init__(self, data_path, sample_size=500):
        self.data_path = data_path
        self.sample_size = sample_size  # 기본값을 500으로 증가
        self.categories_definitions = CATEGORIES_DEFINITIONS
        self.categories = list(CATEGORIES_DEFINITIONS.keys())  # categories 속성 추가
        
        # 데이터 준비
        self.train_df, self.test_df = self._prepare_data()
        
    def _prepare_data(self):
        """파인튜닝용 데이터 준비"""
        df = pd.read_csv(self.data_path)
        
        # 카테고리 컬럼 정규화
        if 'categories' not in df.columns and 'category' in df.columns:
            df = df.rename(columns={'category': 'categories'})
        
        df.dropna(subset=['title', 'categories'], inplace=True)
        df['category'] = df['categories'].apply(lambda x: x.split(';')[0].strip() if isinstance(x, str) else x)
        df = df[df['category'].isin(list(CATEGORIES_DEFINITIONS.keys()))]
        
        # 파인튜닝용 데이터 샘플링
        if len(df) > self.sample_size:
            df = df.sample(n=self.sample_size, random_state=42)
            print(f"📊 파인튜닝용 데이터: {self.sample_size}개 (전체 {len(df)}개에서 샘플링)")
        else:
            print(f"📊 전체 데이터 사용: {len(df)}개")
        
        # 훈련/테스트 분할
        train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
        
        # NER 전처리
        train_df = train_df.copy()
        test_df = test_df.copy()
        train_df['generalized_title'] = ner_generalize_texts(train_df['title'].tolist())
        test_df['generalized_title'] = ner_generalize_texts(test_df['title'].tolist())
        
        print(f"✅ 파인튜닝 데이터 준비: 훈련 {len(train_df)}개, 테스트 {len(test_df)}개")
        return train_df, test_df
    
    def create_training_examples(self):
        """파인튜닝용 InputExample 생성"""
        examples = []
        for _, row in self.train_df.iterrows():
            title = row['generalized_title']
            category = row['category']
            
            # 해당 카테고리의 키워드들과 positive example 생성
            if category in self.categories_definitions:
                for keyword in self.categories_definitions[category]:
                    examples.append(InputExample(texts=[title, keyword]))
        
        print(f"✅ 파인튜닝 예시 생성: {len(examples)}개")
        return examples

print("✅ 파인튜닝 클래스 정의 완료")


In [ ]:
# 🚀 V4 파인튜닝 실행 (조건부)
print("\\n" + "="*80)
print(" " * 15 + "🎓 V4: 파인튜닝 + 앙상블 기법 적용")
print("="*80)

# V3 앙상블 결과가 기본 V3보다 좋은지 확인
v3_baseline = baselines['V3']['hit_rate_1']
best_ensemble_hit1 = best_results.get('hit_rate_1', 0)

print(f"\\n📊 성능 비교:")
print(f"  V3 기본: {v3_baseline:.3f}")
print(f"  V3 앙상블: {best_ensemble_hit1:.3f}")
print(f"  향상도: {((best_ensemble_hit1 - v3_baseline) / v3_baseline * 100):.1f}%")

# 앙상블이 향상되었다면 파인튜닝도 적용
if best_ensemble_hit1 > v3_baseline:
    print("\\n✅ 앙상블 기법이 성능을 향상시켰습니다!")
    print("🔄 V4 파인튜닝을 진행합니다...")
    
    # 파인튜닝 실행 (더 많은 데이터 사용)
    finetuner = QuickFinetuner(DATA_PATH, sample_size=800)  # 전체 데이터의 절반 정도 사용
    
    # 파인튜닝 (더 많은 에포크로 향상된 학습)
    print("\\n🎓 파인튜닝 실행 중... (3 epochs)")
    finetuned_model = finetuner.quick_finetune(epochs=3)
    
    print("✅ V4 파인튜닝 완료!")
    
else:
    print("\\n❌ 앙상블 기법이 큰 향상을 보이지 않았습니다.")
    print("파인튜닝을 건너뜁니다.")
    finetuned_model = None


In [ ]:
# 🔄 파인튜닝 메소드 추가
class QuickFinetuner(QuickFinetuner):  # 클래스 확장
    
    def quick_finetune(self, epochs=1):
        """빠른 파인튜닝 수행"""
        print("🔄 파인튜닝 시작...")
        
        # wandb 비활성화 확인
        os.environ['WANDB_DISABLED'] = 'true'
        
        # 모델 로드
        model = SentenceTransformer(BASE_MODEL_NAME, device=device)
        
        # 특수 토큰 추가
        model.tokenizer.add_special_tokens({"additional_special_tokens": NER_SPECIAL_TOKENS})
        model._first_module().auto_model.resize_token_embeddings(len(model.tokenizer))
        
        # 훈련 예시 생성
        train_examples = self.create_training_examples()
        
        # 데이터로더 생성
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
        
        # 손실 함수
        train_loss = losses.MultipleNegativesRankingLoss(model)
        
        # 훈련 실행
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=epochs,
            warmup_steps=max(1, int(len(train_dataloader) * 0.1)),
            show_progress_bar=True
        )
        
        print("✅ 파인튜닝 완료")
        return model

print("✅ 파인튜닝 메소드 추가 완료")


In [ ]:
# 🎯 V4 앙상블 평가 (파인튜닝된 모델 포함)
if finetuned_model is not None:
    print("\\n" + "="*60)
    print(" " * 15 + "🔥 V4 앙상블 평가")
    print("="*60)
    
    # 파인튜닝된 모델로 V4 앙상블 평가
    class V4EnsembleEvaluator:
        """V4용 앙상블 평가기"""
        
        def __init__(self, test_df, base_model, finetuned_model):
            self.test_df = test_df
            self.base_model = base_model
            self.finetuned_model = finetuned_model
            self.categories = list(CATEGORIES_DEFINITIONS.keys())
            
            # 파인튜닝된 모델의 카테고리 임베딩 계산
            self.finetuned_keyword_embs = self._get_keyword_avg_embs(finetuned_model)
            self.base_keyword_embs = self._get_keyword_avg_embs(base_model)
            
        def _get_keyword_avg_embs(self, model):
            """키워드 평균 임베딩 계산"""
            category_embs = {}
            for category, keywords in CATEGORIES_DEFINITIONS.items():
                keyword_embs = model.encode(keywords, convert_to_numpy=True, normalize_embeddings=True)
                avg_emb = np.mean(keyword_embs, axis=0)
                if np.linalg.norm(avg_emb) > 0:
                    avg_emb = avg_emb / np.linalg.norm(avg_emb)
                category_embs[category] = avg_emb
            return torch.tensor(np.array(list(category_embs.values()))).to(device)
        
        def evaluate_v4_ensemble(self, ensemble_config):
            """V4 앙상블 평가"""
            all_similarities = []
            weights = []
            
            for config in ensemble_config:
                model_type = config['model']
                embedding_type = config['embedding']
                weight = config['weight']
                
                # 모델 선택
                if model_type == 'base':
                    model = self.base_model
                    category_embs = self.base_keyword_embs
                elif model_type == 'finetuned':
                    model = self.finetuned_model
                    category_embs = self.finetuned_keyword_embs
                else:
                    continue
                
                # 텍스트 임베딩
                text_embs = model.encode(
                    self.test_df['generalized_title'].tolist(),
                    convert_to_tensor=True,
                    normalize_embeddings=True
                )
                
                # 유사도 계산
                similarities = util.cos_sim(text_embs, category_embs).cpu().numpy()
                all_similarities.append(similarities)
                weights.append(weight)
            
            # 앙상블 계산
            if not all_similarities:
                return {}
            
            weights = np.array(weights) / np.sum(weights)
            ensemble_similarities = np.zeros_like(all_similarities[0])
            for sim, w in zip(all_similarities, weights):
                ensemble_similarities += sim * w
            
            # 성능 계산
            pred_indices = np.argsort(ensemble_similarities, axis=1)[:, ::-1]
            true_categories = self.test_df['category'].tolist()
            true_indices = [self.categories.index(cat) for cat in true_categories]
            
            correct_at_1 = sum(1 for i, true_idx in enumerate(true_indices) if true_idx == pred_indices[i, 0])
            correct_at_3 = sum(1 for i, true_idx in enumerate(true_indices) if true_idx in pred_indices[i, :3])
            
            total_count = len(self.test_df)
            hit_rate_1 = correct_at_1 / total_count
            hit_rate_3 = correct_at_3 / total_count
            
            # F1 계산
            top_1_predictions = [self.categories[idx] for idx in pred_indices[:, 0]]
            _, _, f1, _ = precision_recall_fscore_support(
                true_categories, top_1_predictions, average='macro', zero_division=0
            )
            
            return {
                'hit_rate_1': hit_rate_1,
                'hit_rate_3': hit_rate_3,
                'f1_score': f1
            }
    
    # V4 평가 실행
    v4_evaluator = V4EnsembleEvaluator(optimizer.test_df, optimizer.base_model, finetuned_model)
    
    # 여러 V4 앙상블 구성 테스트
    v4_configs = [
        # Base + Finetuned 조합
        [
            {'model': 'base', 'embedding': 'keyword_avg', 'weight': 0.4},
            {'model': 'finetuned', 'embedding': 'keyword_avg', 'weight': 0.6}
        ],
        [
            {'model': 'base', 'embedding': 'keyword_avg', 'weight': 0.3},
            {'model': 'finetuned', 'embedding': 'keyword_avg', 'weight': 0.7}
        ],
        [
            {'model': 'base', 'embedding': 'keyword_avg', 'weight': 0.5},
            {'model': 'finetuned', 'embedding': 'keyword_avg', 'weight': 0.5}
        ]
    ]
    
    print("\\n🔍 V4 앙상블 구성 테스트:")
    v4_results = {}
    
    for i, config in enumerate(v4_configs):
        result = v4_evaluator.evaluate_v4_ensemble(config)
        v4_results[f'V4_Config_{i+1}'] = result
        print(f"\\n구성 {i+1}: {config}")
        print(f"결과: Hit@1={result['hit_rate_1']:.3f}, Hit@3={result['hit_rate_3']:.3f}, F1={result['f1_score']:.3f}")
    
    # 최고 V4 결과 찾기
    best_v4 = max(v4_results.items(), key=lambda x: x[1]['hit_rate_1'])
    print(f"\\n🏆 최고 V4 성능: {best_v4[0]} - Hit@1: {best_v4[1]['hit_rate_1']:.3f}")
    
else:
    print("\\n⏭️ 파인튜닝이 수행되지 않아 V4 평가를 건너뜁니다.")


In [ ]:
# 📈 최종 결과 종합 및 시각화
print("\\n" + "="*80)
print(" " * 20 + "🏆 최종 종합 결과")
print("="*80)

# 모든 결과 통합
all_results = {
    'V1 (기본)': baselines['V1'],
    'V2 (NER)': baselines['V2'], 
    'V3 (NER+키워드)': baselines['V3'],
    'V3 앙상블': {
        'hit_rate_1': best_results.get('hit_rate_1', 0),
        'hit_rate_3': best_results.get('hit_rate_3', 0),
        'f1_score': best_results.get('f1_score', 0)
    }
}

# V4 결과가 있다면 추가
if finetuned_model is not None and 'best_v4' in locals():
    all_results['V4 (파인튜닝+앙상블)'] = best_v4[1]

# 최종 성능 표 출력
print("\\n📊 성능 비교표:")
print("-" * 80)
print(f"{'방법':<20} {'Hit@1':<10} {'Hit@3':<10} {'F1-Score':<10} {'향상도':<10}")
print("-" * 80)

baseline_hit1 = baselines['V1']['hit_rate_1']  # V1을 기준으로 향상도 계산

for method, results in all_results.items():
    hit1 = results['hit_rate_1']
    hit3 = results['hit_rate_3'] 
    f1 = results['f1_score']
    improvement = ((hit1 - baseline_hit1) / baseline_hit1 * 100) if baseline_hit1 > 0 else 0
    
    print(f"{method:<20} {hit1:<10.3f} {hit3:<10.3f} {f1:<10.3f} {improvement:<10.1f}%")

print("-" * 80)

# 최종 시각화
plt.figure(figsize=(14, 10))

# 성능 지표별 서브플롯
methods = list(all_results.keys())
hit1_scores = [all_results[m]['hit_rate_1'] for m in methods]
hit3_scores = [all_results[m]['hit_rate_3'] for m in methods]
f1_scores = [all_results[m]['f1_score'] for m in methods]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hit Rate @1
axes[0,0].bar(methods, hit1_scores, color='skyblue', alpha=0.8)
axes[0,0].set_title('Hit Rate @1 비교', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('정확도')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(axis='y', alpha=0.3)

# Hit Rate @3
axes[0,1].bar(methods, hit3_scores, color='lightcoral', alpha=0.8)
axes[0,1].set_title('Hit Rate @3 비교', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('정확도')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(axis='y', alpha=0.3)

# F1 Score
axes[1,0].bar(methods, f1_scores, color='lightgreen', alpha=0.8)
axes[1,0].set_title('F1 Score 비교', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('F1 Score')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(axis='y', alpha=0.3)

# 종합 성능 (Hit@1 기준)
improvements = [((score - baseline_hit1) / baseline_hit1 * 100) if baseline_hit1 > 0 else 0 for score in hit1_scores]
colors = ['red' if imp < 0 else 'green' for imp in improvements]
axes[1,1].bar(methods, improvements, color=colors, alpha=0.7)
axes[1,1].set_title('V1 대비 향상도 (%)', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('향상도 (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
axes[1,1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.suptitle('한국어 텍스트 분류 성능 종합 비교', fontsize=16, fontweight='bold', y=1.02)
plt.show()


In [ ]:
## 🔍 결론 및 인사이트

### 🎯 주요 발견사항

1. **앙상블 기법의 효과**
   - 단일 모델보다 앙상블이 일반적으로 더 안정적인 성능을 보임
   - 가중치 조합이 성능에 중요한 영향을 미침

2. **NER 전처리의 영향**
   - 개체명 인식을 통한 텍스트 일반화가 성능 향상에 기여
   - 특히 인물명, 장소명이 포함된 텍스트에서 효과적

3. **키워드 기반 임베딩**
   - 단순 카테고리명보다 행동 지향적 키워드가 더 효과적
   - 도메인 특화 키워드 설계의 중요성

4. **파인튜닝 효과**
   - 작은 데이터셋에서도 파인튜닝이 성능 개선에 도움
   - 앙상블과 결합 시 상승효과 기대

### 💡 실무 적용 권장사항

1. **단계적 접근**: V1 → V2 → V3 → V4 순으로 점진적 개선
2. **데이터 품질**: 고품질 라벨링과 키워드 설계가 핵심
3. **하이퍼파라미터**: 도메인별 최적 가중치 탐색 필요
4. **리소스 고려**: 성능 대비 계산 비용 트레이드오프 검토

### 🚀 향후 개선 방향

- 더 다양한 앙상블 전략 (stacking, voting 등)
- 대용량 데이터셋에서의 성능 검증
- 실시간 추론 최적화
- 도메인 적응 기법 적용


In [ ]:
# 🎓 파인튜닝 클래스 정의
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import SentenceEvaluator
import csv

class QuickFinetuner:
    """빠른 파인튜닝을 위한 클래스 (Colab 최적화)"""
    
    def __init__(self, data_path, sample_size=50):
        self.data_path = data_path
        self.sample_size = sample_size
        self.categories_definitions = CATEGORIES_DEFINITIONS
        
        # 데이터 준비
        self.train_df, self.test_df = self._prepare_data()
        
    def _prepare_data(self):
        """파인튜닝용 데이터 준비"""
        df = pd.read_csv(self.data_path)
        
        # 카테고리 컬럼 정규화
        if 'categories' not in df.columns and 'category' in df.columns:
            df = df.rename(columns={'category': 'categories'})
        
        df.dropna(subset=['title', 'categories'], inplace=True)
        df['category'] = df['categories'].apply(lambda x: x.split(';')[0].strip() if isinstance(x, str) else x)
        df = df[df['category'].isin(list(CATEGORIES_DEFINITIONS.keys()))]
        
        # 빠른 실행을 위해 작은 샘플 사용
        if len(df) > self.sample_size:
            df = df.sample(n=self.sample_size, random_state=42)
        
        # 훈련/테스트 분할
        train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
        
        # NER 전처리
        train_df = train_df.copy()
        test_df = test_df.copy()
        train_df['generalized_title'] = ner_generalize_texts(train_df['title'].tolist())
        test_df['generalized_title'] = ner_generalize_texts(test_df['title'].tolist())
        
        print(f"✅ 파인튜닝 데이터 준비: 훈련 {len(train_df)}개, 테스트 {len(test_df)}개")
        return train_df, test_df
    
    def create_training_examples(self):
        """파인튜닝용 InputExample 생성"""
        examples = []
        for _, row in self.train_df.iterrows():
            title = row['generalized_title']
            category = row['category']
            
            # 해당 카테고리의 키워드들과 positive example 생성
            if category in self.categories_definitions:
                for keyword in self.categories_definitions[category]:
                    examples.append(InputExample(texts=[title, keyword]))
        
        print(f"✅ 파인튜닝 예시 생성: {len(examples)}개")
        return examples
    
    def quick_finetune(self, epochs=1):
        """빠른 파인튜닝 수행"""
        print("🔄 파인튜닝 시작...")
        
        # 모델 로드
        model = SentenceTransformer(BASE_MODEL_NAME, device=device)
        
        # 훈련 예시 생성
        train_examples = self.create_training_examples()
        
        # 데이터로더 생성
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
        
        # 손실 함수
        train_loss = losses.MultipleNegativesRankingLoss(model)
        
        # 훈련 실행
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=epochs,
            warmup_steps=int(len(train_dataloader) * 0.1),
            show_progress_bar=True
        )
        
        print("✅ 파인튜닝 완료")
        return model

print("✅ 파인튜닝 클래스 정의 완료")
